<a href="https://colab.research.google.com/github/britneytota/weatherwise-Nyaradzo-BritineyTota/blob/main/starter%20notebook%20-britineytota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import re
import dateparser
from datetime import datetime
import matplotlib.pyplot as plt

# Weather API
WEATHER_API_KEY = "040df56c128a4341859193851252205"
BASE_WEATHER_URL = "http://api.weatherapi.com/v1"

def get_weather_data(location, forecast_days=3):
    endpoint = f"{BASE_WEATHER_URL}/forecast.json"
    params = {
        'key': WEATHER_API_KEY,
        'q': location,
        'days': forecast_days,
        'aqi': 'no',
        'alerts': 'no'
    }
    try:
        response = requests.get(endpoint, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"🔴 Error fetching data for {location}: {e}")
        return None

def create_temperature_visualisation(weather_data):
    dates = [day['date'] for day in weather_data['forecast']['forecastday']]
    temps = [day['day']['avgtemp_c'] for day in weather_data['forecast']['forecastday']]

    plt.figure(figsize=(8, 4))
    plt.plot(dates, temps, marker='o', linestyle='--', color='tomato')
    plt.title(f"🌡️ Avg Temperature in {weather_data['location']['name']}")
    plt.xlabel('Date')
    plt.ylabel('Temp (°C)')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def create_precipitation_visualisation(weather_data):
    dates = [day['date'] for day in weather_data['forecast']['forecastday']]
    precipitation = [day['day']['daily_chance_of_rain'] for day in weather_data['forecast']['forecastday']]

    plt.figure(figsize=(8, 4))
    plt.bar(dates, precipitation, color='skyblue')
    plt.title(f"☔ Rain Chances in {weather_data['location']['name']}")
    plt.xlabel('Date')
    plt.ylabel('Rain Chance (%)')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def extract_nlp_info(user_input):
    location_match = re.search(r"in\s+([a-zA-Z\s,]+)", user_input)
    location = location_match.group(1).strip() if location_match else "Mauritius"

    parsed_date = dateparser.parse(user_input, settings={'PREFER_DATES_FROM': 'future'})
    today = datetime.now()
    if parsed_date:
        delta_days = (parsed_date.date() - today.date()).days
        forecast_days = min(max(delta_days + 1, 1), 3)
    else:
        forecast_days = 3

    if "temperature" in user_input.lower() or "hot" in user_input.lower():
        intent = "temperature"
    elif "rain" in user_input.lower() or "precipitation" in user_input.lower():
        intent = "rain"
    else:
        intent = "both"

    return location, forecast_days, intent

def ai_weather_assistant():
    print("🤖 Hello! I'm your Weather AI Assistant.")
    print("🔍 Ask me anything about the weather, like:")
    print("   - Will it rain in Paris tomorrow?")
    print("   - Show me the temperature in London next 3 days.")
    print("   - What's the weather like in Tokyo?\n")
    print("💬 Type 'exit' to quit.\n")

    while True:
        user_input = input("🗣️  You: ").strip()
        if user_input.lower() in ['exit', 'quit']:
            print("👋 Goodbye! Stay safe and weather-ready!")
            break

        location, forecast_days, intent = extract_nlp_info(user_input)
        print(f"📡 Fetching forecast for {location} ({forecast_days} day(s))...")

        data = get_weather_data(location, forecast_days)
        if not data:
            print("❌ Could not retrieve weather data. Please try a different location.")
            continue

        if intent == "temperature":
            print("📊 Here's the temperature forecast:")
            create_temperature_visualisation(data)
        elif intent == "rain":
            print("🌧️ Here's the rain forecast:")
            create_precipitation_visualisation(data)
        else:
            print("📊 Temperature and ☔ Rain forecast coming up...")
            create_temperature_visualisation(data)
            create_precipitation_visualisation(data)

if __name__ == "__main__":
    ai_weather_assistant()

